In [1]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
!pip install google-api-python-client==1.7.2
!pip install google-auth==1.8.0
!pip install google-auth-httplib2==0.0.3
!pip install google-auth-oauthlib==0.4.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 1.4.6 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.7.2 which is incompatible.
pydrive2 1.21.3 requires google-api-python-client>=1.12.5, but you have google-api-python-client 1.7.2 which is incompatible.
firebase-admin 6.6.0 requires google-api-python-client>=1.7.8, but you have google-api-

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.1
    Uninstalling cachetools-5.5.1:
      Successfully uninstalled cachetools-5.5.1
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 1.8.0 which is incompatible.
google-cloud-functions 1.19.0 requires google-auth!=2.24.0,!=2.25.0,<3.0.0dev,>=2.14.1, but you have google-auth 1.8.0 which is incompatible.
google-generativeai 0.8.4

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [29]:
from google.colab import auth
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import os

# Authentication
auth.authenticate_user()

# Define SCOPES
SCOPES = ['https://www.googleapis.com/auth/documents']

# Load credentials and create a Google Docs API service
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=33965)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Create the Google Docs API service
service = build('docs', 'v1', credentials=creds)

# Provided markdown content
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

def create_google_doc_from_markdown(markdown_content):
    # Create a new Google Doc
    doc = service.documents().create().execute()
    document_id = doc['documentId']

    # Initialize list to hold the requests for document formatting
    requests = []

    # Split the markdown content and process line by line
    lines = markdown_content.splitlines()
    current_index = 1  # Start at position 1 in the document

    for line in lines:
        if line.startswith('# '):  # Heading 1
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line[2:] + '\n'}
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(line[2:]) + 1},
                    'paragraphStyle': {'namedStyleType': 'HEADING_1'}
                }
            })
            current_index += len(line) + 1

        elif line.startswith('## '):  # Heading 2
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line[3:] + '\n'}
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(line[3:]) + 1},
                    'paragraphStyle': {'namedStyleType': 'HEADING_2'}
                }
            })
            current_index += len(line) + 1

        elif line.startswith('### '):  # Heading 3
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line[4:] + '\n'}
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': current_index, 'endIndex': current_index + len(line[4:]) + 1},
                    'paragraphStyle': {'namedStyleType': 'HEADING_3'}
                }
            })
            current_index += len(line) + 1

        elif line.startswith('- [ ]'):  # Checkbox
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line + '\n'}
            })
            current_index += len(line) + 1

        elif line.startswith('* '):  # Bullets
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line[2:] + '\n'}
            })
            current_index += len(line) + 1

        else:  # Regular text
            requests.append({
                'insertText': {'location': {'index': current_index}, 'text': line + '\n'}
            })
            current_index += len(line) + 1

    # Send the requests to Google Docs API
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

    return document_id

# Create the Google Doc from markdown content
doc_id = create_google_doc_from_markdown(markdown_content)

print(f"Document created at: https://docs.google.com/document/d/{doc_id}/edit")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=429426087425-l9f8tfahed3gediorvg2sfcr1clmg1ue.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A33965%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments&state=AA9YDGPfIEG8MsVFjLQQFKrwhrchgb&access_type=offline


KeyboardInterrupt: 